<a href="https://colab.research.google.com/github/Muhammad-javed440/Q-3/blob/master/tool_calling_color_light.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tool Calling | Function calling

In [43]:
!pip install -qU langchain_google_genai

In [44]:
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import display, Markdown
import google.generativeai as genai

In [45]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')

# Creating Models

In [46]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    api_key=GOOGLE_API_KEY,
    temperature=0.4,
    max_tokens=100,
    max_retries=3,
)

In [47]:
response =llm.invoke("What is the capital of pakistan?")
display(Markdown(response.content))

The capital of Pakistan is **Islamabad**.

# Creating tools

In [64]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')

In [65]:
genai.configure(api_key=GOOGLE_API_KEY)

In [66]:
from langchain_core.tools import tool
import requests

def enable_light()->str:  # Removed @tool decorator
  """Turn on the lighting system"""
  print("LIGHT BOT: Lighting system turned on")
  return "Lighting system turned on"  # Added return statement

def set_light_color(rgb_hex:str)->str:  # Removed @tool decorator
  """ Set the light color ,Light must be enabled for this to work"""
  print(f"LIGHT BOT:light set to{rgb_hex}")
  return f"Light set to {rgb_hex}"  # Added return statement

def disable_light()->str:  # Removed @tool decorator
  """Turn off the lighting system"""
  print("LIGHT BOT: Lighting system turned off")
  return "Lighting system turned off"  # Added return statement

# Convert Langchain tools to Google Generative AI tools
google_ai_tools = [
    content_types.CallableFunctionDeclaration.from_function(func)
    for func in [enable_light, set_light_color, disable_light]
]

instruction = """You are helpful lighting system bot.You can lights on and off, you can set the color .Do not perform any other tasks. """

# Pass the converted tools to GenerativeModel
model = genai.GenerativeModel(
    "models/gemini-2.0-flash-exp",
    tools=google_ai_tools, # Use converted tools
    system_instruction=instruction
)

chat = model.start_chat()

In [71]:
from google.generativeai.types import content_types
from collections.abc import Iterable


# Create a helper function for setting " function_calling_config " on " tool_config "

In [72]:
def tool_config_from_mode(mode:str, fns: Iterable[str]=()):
  """Create a tool config with the specified function calling mode."""
  return content_types.to_tool_config(
      {
      "function_calling_config":{"mode":mode, "allowed_function_names":fns}
      }
      )

# Text_only mode:NONE
### If you have provided themodel with tools, but do not want to use those tools for current conversationalturn, then specify
### NONE as the mode.NONE tells the model not to make any function calls, and will behave as though none havebeen provided.


In [73]:
tool_config = tool_config_from_mode("none")
response = chat.send_message(
    "Hello light_bot, what can you do?",
    tool_config = tool_config
)
print(response.text)


Hello! I can turn lights on and off, and I can set their color.



# Automatic mode : AUTO

To allow the model to decide whether to respond in text or call specific functions, you can specify AUTO as the mode.

In [74]:
tool_config = tool_config_from_mode("auto")

response = chat.send_message(
    "Light this place up!",
    tool_config = tool_config
)
print(response.parts[0])
chat.rewind();

function_call {
  name: "enable_light"
  args {
  }
}



# Function-calling mode : ANY

Setting the mode to ANY will force the model to make a function call. By setting 'allowed_function_names' , In this example , If the lights are already on, then the user can change color or turn the lights off.


In [75]:
available_fns =["set_light_color","disable_light"]
tool_config = tool_config_from_mode("any", available_fns)

response = chat.send_message(
    "Make this place PURPLE!",
    tool_config = tool_config
)
print(response.parts[0])

function_call {
  name: "set_light_color"
  args {
    fields {
      key: "rgb_hex"
      value {
        string_value: "800080"
      }
    }
  }
}



# Automatic function calling

'tool_config' works when enabling automatic function calling too.

In [79]:
available_fns = ["enable_light"]
tool_config = tool_config_from_mode("any", available_fns)

auto_chat = model.start_chat(enable_automatic_function_calling=True)
response=auto_chat.send_message("It's a awful dark in here...", tool_config=tool_config)

LIGHT BOT: Lighting system turned on


In [81]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "That should be better!\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            }
          ],
          "avg_logprobs": -0.23144346475601196
        }
      ],
      "usage

# Run all the function

In [85]:
available_fns = ["enable_light","set_light_color","disable_light"]
tool_config = tool_config_from_mode("any", available_fns)

In [88]:
auto_chat = model.start_chat(enable_automatic_function_calling=True)
response=auto_chat.send_message("enable lights and set light color red and then disable light", tool_config=tool_config)

LIGHT BOT: Lighting system turned on
LIGHT BOT:light set toFF0000
LIGHT BOT: Lighting system turned off


In [89]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "OK. I've enabled the lights, set the color to red, and then disabled the lights. Anything else?\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            }
          ],
     

# Lights off

In [90]:
auto_chat = model.start_chat(enable_automatic_function_calling=True)
response=auto_chat.send_message(" disable light", tool_config=tool_config)

LIGHT BOT: Lighting system turned off


In [91]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "OK. I've turned off the lights.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            }
          ],
          "avg_logprobs": -0.011215082623741844
        }
      ],
  

# Creating a function Set the brightness of a room

In [92]:
{
  "brightness": "50",
  "colorTemperature": "daylight"
}

{'brightness': '50', 'colorTemperature': 'daylight'}

In [104]:
from typing import Literal
def set_light_values(brightness:int, color_temp: Literal["daylight","cool","warm"]):
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {
        "brightness": brightness,
        "colorTemperature": color_temp
    }

In [105]:
model = genai.GenerativeModel(model_name='gemini-2.0-flash-exp',
                              tools=[set_light_values])

In [112]:
chat =model.start_chat()
response1 = chat.send_message('Dim the lights so the room feels cozy and warm.')

In [119]:
response1

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "set_light_values",
                  "args": {
                    "brightness": 30.0,
                    "color_temp": "warm"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM

# Music function calling

In [122]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

In [123]:
# Set the model up with tools.
house_fns = [power_disco_ball, start_music, dim_lights]

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=house_fns)

# Call the API.
chat = model.start_chat()
response = chat.send_message("Turn this place into a party!")

# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

power_disco_ball(power=True)
start_music(loud=True, energetic=True, bpm=120.0)
dim_lights(brightness=0.5)


In [124]:
# Simulate the responses from the specified tools.
responses = {
    "power_disco_ball": True,
    "start_music": "Never gonna give you up.",
    "dim_lights": True,
}

# Build the response parts.
response_parts = [
    genai.protos.Part(function_response=genai.protos.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

response = chat.send_message(response_parts)
print(response.text)

Party started!  Playing "Never gonna give you up" at 120 bpm.  The disco ball is spinning and the lights are dimmed to 50% brightness.



In [125]:
model._tools.to_proto()

[function_declarations {
   name: "power_disco_ball"
   description: "Powers the spinning disco ball."
   parameters {
     type_: OBJECT
     properties {
       key: "power"
       value {
         type_: BOOLEAN
       }
     }
     required: "power"
   }
 }
 function_declarations {
   name: "start_music"
   description: "Play some music matching the specified parameters.\n\n    Args:\n      energetic: Whether the music is energetic or not.\n      loud: Whether the music is loud or not.\n      bpm: The beats per minute of the music.\n\n    Returns: The name of the song being played.\n    "
   parameters {
     type_: OBJECT
     properties {
       key: "loud"
       value {
         type_: BOOLEAN
       }
     }
     properties {
       key: "energetic"
       value {
         type_: BOOLEAN
       }
     }
     properties {
       key: "bpm"
       value {
         type_: INTEGER
       }
     }
     required: "energetic"
     required: "loud"
     required: "bpm"
   }
 }
 functi